# Error Studies with Parallelised Pynac
Now that I've demonstrated how to parallelise multiple instances of Pynac, I want to show how it might be used for error studies.

Pynac is currently in a file in the local directory, and so can be directly imported.

In [1]:
from pynac import Pynac, multiProcessPynac, PhaseSpace, makePhaseSpaceList, getNumberOfParticles
from elements import Quad, CavityAnalytic
import os
import glob
import shutil
import random
from contextlib import contextmanager
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
output_notebook()

def clean():
    for outputdir in glob.glob('./dynacProc_*'):
        if os.path.isdir(outputdir):
            shutil.rmtree(outputdir)

clean()

Loading BokehJS ...

## A function to do the hard work
Dynac atuomatically creates a lot of output files, so it is wise to create new directories to run each instance in in order to prevent overwriting of the data.  The following function creates a new folder (if it finds one with the same name, it automatically deletes it!!), and copies the relevant files there.  It then changes the working directory to that folder, and executes Pynac.

More importantly, it also adds some errors.

In [2]:
random.seed(197812160518)

def getRandomScaling(scale = 1e-3):
#     return scale * ((random.random()*2) - 1)
    return random.gauss(0, scale)

def applyErrorsAndRunPynac():
    a = Pynac('ESS_with_SC_ana.in')
    for ind in a.getXinds('QUADRUPO'):
        quad = Quad(a.lattice[ind])
        quad.scaleField(1 + getRandomScaling(scale = 1e-2))
        a.lattice[ind] = quad.dynacRepresentation()
    for ind in a.getXinds('CAVMC'):
        cav = CavityAnalytic(a.lattice[ind])
        cav.adjustPhase(getRandomScaling(2.0))
        cav.scaleField(1 + getRandomScaling(scale = 1e-3))
        a.lattice[ind] = cav.dynacRepresentation()
    a.run()

## Running in parallel
Now run some randomly generated errors in random.

In [3]:
%%time
filelist =['ESS_with_SC_ana.in', 
           'ESS_RFQ_out_70mA.dst', 
           'Spoke_F2F_field.txt', 
           'MBL_F2F_field.txt', 
           'HBL_F2F_field.txt']
print(multiProcessPynac(filelist = filelist, pynacFunc = applyErrorsAndRunPynac, numIters = 400, max_workers = 8))

Running 0
Running 3
Running 6
Running 1
Running 2
Running 4
Running 5
Running 7
Running 8
Running 9
Running 10
Running 11
Running 12
Running 13
Running 14
Running 15
Running 16
Running 17
Running 18
Running 19
Running 20
Running 21
Running 22
Running 23
Running 24
Running 25
Running 26
Running 27
Running 28
Running 29
Running 30
Running 31
Running 32
Running 33
Running 34
Running 35
Running 36
Running 38
Running 37
Running 39
Running 40
Running 42
Running 41
Running 44
Running 43
Running 45
Running 46
Running 47
Running 48
Running 49
Running 50
Running 51
Running 52
Running 53
Running 54
Running 55
Running 56
Running 57
Running 58
Running 59
Running 60
Running 61
Running 62
Running 63
Running 64
Running 66
Running 65
Running 67
Running 68
Running 69
Running 70
Running 71
Running 72
Running 73
Running 74
Running 75
Running 76
Running 77
Running 78
Running 79
Running 80
Running 81
Running 82
Running 83
Running 84
Running 85
Running 86
Running 87
Running 88
Running 89
Running 90
Running 9

# Analysis
So far, in building Pynac, we haven't yet done any analysis of the output beyond looking at the standard plots produced by `Pynplot.plotit()`.  The Dynac run produces many other output files containing significant information, and so now we start delveoping a class to look at this.

In [4]:
from bokeh.models.ranges import Range1d
from bokeh.models.axes import LinearAxis
import numpy as np

@contextmanager
def workInOtherDirectory(dirname):
    presentDir = os.getcwd()
    os.chdir(dirname)
    try:
        yield
    finally:
        os.chdir(presentDir)

p = figure(plot_width=800, plot_height=400)
partLeftAtEnd = []
for dirnum in range(9999):
    try:
        with workInOtherDirectory('dynacProc_%04d' % dirnum):
            psList = makePhaseSpaceList()
            xemit = [ps.xPhaseSpace.normEmit.val for ps in psList]
            numParts = getNumberOfParticles()
            partLeft = [ps.particlesLeft.val/numParts for ps in psList]
            partLeftAtEnd.append(partLeft[-1])
            p.line(range(len(xemit)), xemit)
    except FileNotFoundError:
        break
show(p)

hist, edges = np.histogram(partLeftAtEnd, density=True, bins=50)

In [5]:
import numpy as np
hist, edges = np.histogram(partLeftAtEnd, density=True, bins=0.01 + np.arange(0, 1, 0.01))
p = figure(plot_width=800, plot_height=400)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:])
show(p)

In [6]:
print(os.getcwd())
os.chdir('/Users/stephenmolloy/git/Pynac')
print(os.getcwd())

/Users/stephenmolloy/git/Pynac
/Users/stephenmolloy/git/Pynac
